In [ ]:
import pandas as pd

train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test.csv', encoding='utf-8')
train.head()

In [ ]:
media_id = pd.concat([train.media_id, test.media_id])

In [ ]:
mfcc = []
mel = []
rank = []
bpm = []
gain = []
explicit_lyrics = []

In [ ]:
import json, requests
import librosa
from array import array
from six.moves.urllib.request import urlretrieve
import six.moves.urllib as urllib
from time import sleep

url = 'https://api.deezer.com/track/'
total = len(media_id.unique())
filename = 'test.mp3'
for i,el in enumerate(media_id.unique()) :
    url1 = url + str(el)
    resp = requests.get(url=url1)
    data = json.loads(resp.text)
    preview = data['preview']
    if preview == '':
        if not data.get('alternative'):
            print('no track preview!')
            rank.append(data['rank'])
            bpm.append(data['bpm'])
            gain.append(data['gain'])
            explicit_lyrics.append(data['explicit_lyrics'])
            mel.append([0])
            mfcc.append([0])
            continue
        preview = data['alternative']['preview']
        rank.append(data['alternative']['rank'])
        bpm.append(data['alternative']['bpm'])
        gain.append(data['alternative']['gain'])
        explicit_lyrics.append(data['alternative']['explicit_lyrics'])
        
    else:
        rank.append(data['rank'])
        bpm.append(data['bpm'])
        gain.append(data['gain'])
        explicit_lyrics.append(data['explicit_lyrics'])
    
    try:
        urlretrieve(preview, filename=filename)
        y, sr = librosa.load(filename)
        mel.append(librosa.feature.melspectrogram(y=y, sr=sr))
        mfcc.append(librosa.feature.mfcc(y=y, sr=sr))
    except (urllib.error.HTTPError,urllib.error.URLError) as err:
        mel.append([0])
        mfcc.append([0])
    print("Completed " + str(i) + " from " + str(total))
 

In [ ]:
from multiprocessing import Pool

def process(x):
    url = 'https://api.deezer.com/track/' + str(x)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    preview = data['preview']
    print('Iter')
    if preview == '':
        preview = data['alternative']['preview']
        rank = data['alternative']['rank']
        bpm = data['alternative']['bpm']
        gain = data['alternative']['gain']
        explicit_lyrics = data['alternative']['explicit_lyrics']
        urlretrieve(preview, filename=filename)
        y, sr = librosa.load(filename)
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        return {'id' : x, 'rank': rank, 'bpm': bpm, 'gain': gain, 'explicit_lyrics' : explicit_lyrics, 'mel' : mel, 'mfcc' : mfcc}
        
    else:
        rank = data['rank']
        bpm = data['bpm']
        gain = data['gain']
        explicit_lyrics = data['explicit_lyrics']
        urlretrieve(preview, filename=filename)
        y, sr = librosa.load(filename)
        mel = librosa.feature.melspectrogram(y=y, sr=sr)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        return {'id' : x, 'rank': rank, 'bpm': bpm, 'gain': gain, 'explicit_lyrics' : explicit_lyrics, 'mel' : mel, 'mfcc' : mfcc}
        

pool = Pool()

result = pool.map(process, media_id.unique())

In [ ]:
bpm